In [2]:
import requests
import pandas

import datetime_translator

brandIds = { 202, 88, 31, 123, 101, 122, 36, 48, 135 }
data = {}

for brandId in brandIds:
    headers = { 'User-Agent': '', 'content-type':  'application/json' }
    jsonData = '{"variables":{"area":"salt-lake","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId brandingType imageUrl name  } latitude longitude fuels id name prices(fuel: $fuel) { cash { nickname postedTime price  } credit { nickname postedTime price  } discount fuelProduct } } } } }"}' %(brandId)
    #jsonData = '{"variables":{"area":"davis","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId brandingType imageUrl name  } latitude longitude fuels id name prices(fuel: $fuel) { cash { nickname postedTime price  } credit { nickname postedTime price  } discount fuelProduct } } } } }"}' %(brandId)
    #jsonData = '{"variables":{"area":"utah","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brandId brandingType imageUrl name  } latitude longitude fuels id name prices(fuel: $fuel) { cash { nickname postedTime price  } credit { nickname postedTime price  } discount fuelProduct } } } } }"}' %(brandId)
    response = requests.post('https://www.gasbuddy.com/graphql', headers=headers, data=jsonData)
    jsonResponse = response.json()
    stations = jsonResponse['data']['locationByArea']['stations']

    for station in stations['results']:
        if int(station['brands'][0]['brandId']) in brandIds and (station['prices'][0]['credit']['price'] != 0):
            stationId = station['id']
            data[stationId] = {
                'StationName': station['name'],
                'BrandName': station['brands'][0]['name'],
                'AddressLine1': station['address']['line1'],
                'City': station['address']['locality'],
                'RegularFuelPrice': '${:.2f}'.format(station['prices'][0]['credit']['price']),
                'TimeSinceReported': datetime_translator.translate(station['prices'][0]['credit']['postedTime']),
                'ReportedBy': station['prices'][0]['credit']['nickname']
            }

output = pandas.DataFrame(data)
output.transpose().sort_values(by='RegularFuelPrice')

,StationName,BrandName,AddressLine1,City,RegularFuelPrice,TimeSinceReported,ReportedBy
108136,Shell,Shell,1881 California Ave,Salt Lake City,$4.07,2 hours,poletransit45
159136,Maverik,Maverik,1445 S. Gustin Road,Salt Lake City,$4.09,17 minutes,Owner
191646,Maverik,Maverik,1290 S. Wallace Road,SaltLake,$4.09,18 minutes,Owner
38376,Texaco,Holiday Oil (UT),4783 W 6200 S,West Jordan,$4.09,1 day,smithmarty
62465,Chevron,Chevron,4607 W 6200 S,West Jordan,$4.13,3 hours,love2savemoney2
...,...,...,...,...,...,...,...
107212,Holiday,Holiday Oil (UT),10419 S 4000 W,South Jordan,$4.39,13 hours,Jon4958
21893,Chevron,Chevron,365 W 7200 S,Midvale,$4.39,4 hours,abk
100188,Holiday,Holiday Oil (UT),4391 W 3500 S,West Valley City,$4.39,13 hours,msmith17
62526,Holiday,Holiday Oil (UT),1033 W South Jordan Pkwy,South Jordan,$4.39,5 hours,cspohn
